Importing Libraries and Dependencies

In [85]:
# Mohammad Malik & Adair Maynard
# Task 5: MAMI
# CMSC 516: NLP at VCU, Dr. Bridget McInnes

!pip install utils
!pip install setuptools
!pip install numpy

import os
import io
import utils
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import floor

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import tensorflow as tf
import transformers as trfs
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

# from google.colab import auth
# from googleapiclient.discovery import build
# from io import FileIO
# from googleapiclient.http import MediaIoBaseDownload

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from string import punctuation 
from nltk.stem import WordNetLemmatizer

tf.get_logger().setLevel('ERROR')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [86]:
print(trfs.__version__)
tf.__version__

4.12.5


'2.6.0'

In [87]:
#  Get the files from the google drive
# auth.authenticate_user()
# drive_service = build('drive', 'v3')

# # Get data file
# file_id = '1sgE4tBtZwE-M9Y6rfLu_FlwI6VpnaUo-'  # Training file on the Google Drive
# downloaded = io.FileIO("training.csv", 'w')
# request = drive_service.files().get_media(fileId=file_id)
# downloader = MediaIoBaseDownload(downloaded, request)
# done = False
# while done is False:
#   status, done = downloader.next_chunk()
#   print("Download {}%.".format(int(status.progress() * 100)))

In [88]:
#  Load Tokenizer
# TODO look at the finetuned model because it resulted in a lot of UNKNOWN words for capitals and doesnt know a lot of words
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") #Hate-speech-CNERG/dehatebert-mono-english

# Load Model
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased") #Hate-speech-CNERG/dehatebert-mono-english

In [89]:
# Load dataset
df = pd.read_csv("TRAINING/training.csv", delimiter="\t")

df.head()

,file_name,misogynous,shaming,stereotype,objectification,violence,Text Transcription
0,1.jpg,0,0,0,0,0,Milk Milk.zip
1,10.jpg,1,0,0,0,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S..."
2,1000.jpg,0,0,0,0,0,BREAKING NEWS: Russia releases photo of DONALD...
3,10000.jpg,0,0,0,0,0,MAN SEEKING WOMAN Ignad 18 O
4,10006.jpg,0,0,0,0,0,Me explaining the deep lore of. J.R.R. Tolkein...


In [90]:
# get rid of multi classes and only keep misogyny
X = df.drop(columns=['shaming', 'objectification', 'stereotype', 'violence'])
X

,file_name,misogynous,Text Transcription
0,1.jpg,0,Milk Milk.zip
1,10.jpg,1,"ROSES ARE RED, VIOLETS ARE BLUE IF YOU DON'T S..."
2,1000.jpg,0,BREAKING NEWS: Russia releases photo of DONALD...
3,10000.jpg,0,MAN SEEKING WOMAN Ignad 18 O
4,10006.jpg,0,Me explaining the deep lore of. J.R.R. Tolkein...
...,...,...,...
9995,15002.jpg,0,WAITING FOR THE END OF THE COVID imgflip.com
9996,15003.jpg,0,SMART WOMEN ARE AROUND imgflip.com
9997,15004.jpg,0,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com
9998,15005.jpg,0,COOKING FOR MY WIFE imgflip.com


In [91]:
# preprocess text then pass into model

# download NLTK stopwords, punctuation and wordLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# clean/preprocess dataset instances
def clean_data(trans):
  words = stopwords.words('english')
  filtered_words = " ".join([w for w in trans.split() if not w.lower() in words]) # might need to be trans.lower().split()
  url = ''.join(re.sub(r'http\S+', '', filtered_words))
  almost = ''.join(re.sub(r'www\S+', '', url))
  cleaned = ''.join(re.sub(r'(\w+\.\w+)', '', almost))
  punct = ''.join(ch for ch in cleaned if ch not in punctuation)
  wordnet_lemmatizer = WordNetLemmatizer()

  word_tokens = nltk.word_tokenize(punct)
  lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]

  word_joined = " ".join(lemmatized_word)
  
  return word_joined

[nltk_data] Downloading package stopwords to /Users/mani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [92]:
# apply preprocessing to dataset
X['Cleaned Text'] = X['Text Transcription'].apply(clean_data)

In [93]:
# create new variable that solely consists of cleaned text and labels
X_bert = pd.DataFrame({
    # 'File_Name':X['file_name'],
    'Misogynous':X['misogynous'],
    # 'Alpha':['a']*X.shape[0],
    'Cleaned_Text':X['Cleaned Text']
})
X_bert

,Misogynous,Cleaned_Text
0,0,Milk
1,1,ROSES RED VIOLETS BLUE SAY YES ILL RAPE
2,0,BREAKING NEWS Russia release photo DONALD TRUM...
3,0,MAN SEEKING WOMAN Ignad 18
4,0,explaining deep lore of R Tolkeins world Arda ...
...,...,...
9995,0,WAITING END COVID
9996,0,SMART WOMEN AROUND
9997,0,GOOD GIRLS BEHIND CORNER
9998,0,COOKING WIFE


In [94]:
# get list of sentences and labels
sentences = X_bert.Cleaned_Text.values
print(sentences.shape)

labels = X_bert.Misogynous.values
print(labels)

(10000,)
[0 1 0 ... 0 0 0]


In [95]:
# split dataset into train and test to be passed into models, 85% and 15%
X_train, X_rem, y_train, y_rem = train_test_split(sentences, labels, train_size=0.85)

In [96]:
# split dataset further into valid and test
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, train_size=0.333)

In [97]:
# print(X_train.shape)
# print(y_train.shape)

# print(X_valid.shape)
# print(y_valid.shape)

# print(X_test.shape)
# print(y_test.shape)

In [98]:
# testing bert tokenizer
print('orig: ', X_train[1])
print('Tokenized: ', tokenizer.tokenize(X_train[1]))
print("Token IDs: ", tokenizer.convert_tokens_to_ids(tokenizer.tokenize(X_train[1])))

orig:  REDDIT CAKE DAY ALMOST DONT POST ANYTHING excuse fuck almost forgot post
Tokenized:  ['red', '##dit', 'cake', 'day', 'almost', 'don', '##t', 'post', 'anything', 'excuse', 'fuck', 'almost', 'forgot', 'post']
Token IDs:  [2417, 23194, 9850, 2154, 2471, 2123, 2102, 2695, 2505, 8016, 6616, 2471, 9471, 2695]


In [99]:
# for idx in X_train:
#   # Tokenize the text and add `[CLS]` and `[SEP]` tokens
#   input_ids = tokenizer.encode(idx, add_special_tokens=True)
#   # Update the maximum sentence length
#   max_length = max(max_length, len(input_ids))

# # add 35 incase any instance in test or valid set are larger, could catch some issues, also cuz it rounds to 300
# max_length = max_length + 35

# print(max_length)

In [100]:
MAX_LENGTH = 300

BATCH_SIZE = 256 # 32

EVAL_BATCH_SIZE = BATCH_SIZE * 2

In [101]:
def batch_encode(tokenizer, texts, batch_size=BATCH_SIZE, max_length=MAX_LENGTH):

  input_ids_batch = []
  attention_mask = []

  for i in range(0, len(texts), batch_size): 
    batch = texts[i:i+batch_size]
    
    inputs = tokenizer.batch_encode_plus(
      batch,
      max_length=MAX_LENGTH, # set the length of the sequences
      add_special_tokens=True, # add [CLS] and [SEP] tokens
      return_attention_mask=True,
      truncation=True, # yells if not set but padding is done
      return_token_type_ids=False, # not needed for this type of ML task
      pad_to_max_length=True, # add 0 pad tokens to the sequences less than max_length
      return_tensors='np'
      )
    input_ids_batch.extend(inputs['input_ids'])
    attention_mask.extend(inputs['attention_mask'])
  
  return tf.convert_to_tensor(input_ids_batch), tf.convert_to_tensor(attention_mask)

In [102]:
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.tolist())

# Encode X_valid
X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_valid.tolist())

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.tolist())

/Users/mani/miniforge3/envs/cloud/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [103]:
X_train_ids = np.asarray(X_train_ids)
X_train_attention = np.asarray(X_train_attention)
X_valid_ids = np.asarray(X_valid_ids)
X_valid_attention = np.asarray(X_valid_attention)
X_test_ids =  np.asarray(X_test_ids)
X_test_attention = np.asarray(X_test_attention)
# input_ids = np.asarray(input_ids)
# attention_mask = np.asarray(attention_mask)

In [104]:
def create_model(transformer, max_length=MAX_LENGTH):    
    # Define input layers
    
    # This is the input for the tokens themselves(words from the dataset after encoding):
    input_ids_layer = tf.keras.layers.Input(shape=(max_length,), 
                                            name='input_ids', 
                                            dtype='int32')

    # attention_mask - is a binary mask which tells BERT which tokens to attend and which not to attend.
    # Encoder will add the 0 tokens to the some sequence which smaller than MAX_SEQUENCE_LENGTH, 
    # and attention_mask, in this case, tells BERT where is the token from the original data and where is 0 pad token:

    input_attention_layer = tf.keras.layers.Input(shape=(max_length,), 
                                                  name='input_attention', 
                                                  dtype='int32')
    # Use previous inputs as BERT inputs:
    bert = transformer([input_ids_layer, input_attention_layer])[0]

    # We can also add dropout as regularization technique:
    #output = tf.keras.layers.Dropout(rate=0.15)(output)

    # Provide number of classes to the final layer:
    output = tf.keras.layers.Dense(1, activation='softmax')(bert)

    # Final model:
    model = tf.keras.models.Model([input_ids_layer, input_attention_layer], outputs=output)


    opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [105]:
# bert_model = trfs.TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
bert_model = trfs.AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
from transformers import TFDistilBertModel, DistilBertConfig

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2, output_hidden_states=False)
# bert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [106]:
model = create_model(bert_model, MAX_LENGTH)

AttributeError: 'list' object has no attribute 'size'

In [ ]:
model.summary()

In [ ]:
X_train_ids = np.asarray(X_train_ids)
X_train_attention = np.asarray(X_train_attention)
# X_valid_ids = np.asarray(X_valid_ids)
# X_valid_attention = np.asarray(X_valid_attention)
print(type(X_train_ids))
print(type(X_valid_attention))

In [ ]:
model.fit(
    x=[X_train_ids, X_train_attention],
    y=y_train,
    epochs=3,
    batch_size=BATCH_SIZE,
    validation_data=([X_valid_ids, X_valid_attention], y_valid)
)

In [ ]:
#Create and train the model

#Create LSTM concurrent NN
#train the model
       
#fix random seed for reproducibility
np.random.seed(7)

#load the dataset but only keep the top n words, zero the rest
top_words = 5000
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

#truncate and pad input sequences
# max_review_length = 500
# X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
# X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

#create model
# model = Sequential()
# model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(LSTM(100))
# model.add(Dense(1, activation='sigmoid'))

In [ ]:
# instead of doing it that way let's also see the lstm
def use_lstm(top_words=top_words):
  embedding_vector_length = 32
  model = Sequential()
  model.add(Embedding(top_words, embedding_vector_length, input_length=MAX_LENGTH))
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(LSTM(100))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #need to write code for AUROC, F1, and precision
  print(model.summary())
  model.fit(X_train, y_train, epochs=3, batch_size=64) #may consider changing # of epochs for speed
  # model.train() Error: AttributeError: 'Sequential' object has no attribute 'train'

In [ ]:
model = use_lstm()

In [ ]:
yhat_probs = model.predict(X_test, verbose=0)

In [ ]:
yhat_classes = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
print(yhat_classes)

In [ ]:
yhat_classes = model.predict_classes(X_test, verbose=0) #AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

In [ ]:
#Performance Metrics: Accuracy, precision, recall, F1, cohen's kappa, AUROC, and confusion matrix

# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# predict classes for test set
# yhat_classes = model.predict_classes(X_test, verbose=0) #AttributeError: 'Sequential' object has no attribute 'predict_classes'
yhat_classes = (model.predict(X_test) > 0.5).astype("int32")

# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

# tp = true positive, tn = true negative, fp = false positive, fn = false negative, p = positive, n = negative
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)

# cohen's kappa
kappa = cohen_kappa_score(y_test, yhat_classes)

print('Cohens kappa: %f' % kappa)
# AUROC
auc = roc_auc_score(y_test, yhat_probs)
print('AUROC: %f' % auc)

# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print('Confusion matrix: ', matrix)